In [78]:
import sys
sys.path.append("..")

In [219]:
import nltk, re, string, time, os
import pandas as pd
import numpy as np
# from textClassification import textAnalysis
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet as wn
from sklearn.metrics import accuracy_score
from sklearn import metrics 
from sklearn import cross_validation
import operator
english_stopwords = stopwords.words('english')

### Helper functions

In [81]:
# automate creation of csv for kaggle submission
def create_submission(filename, prediction):
    pd.DataFrame.to_csv(pd.DataFrame(df_test['Id']).join(pd.DataFrame(prediction)), path_or_buf=filename, header=['Id','Category'], index=False)

In [82]:
# yell outloud 'process complete'
def finished():
    os.system("say 'process complete'")

In [83]:
# automate crossvalidation and printing of results
def crossvalidation(model, train_features, real_category):
    scores = cross_validation.cross_val_score(model, train_features, real_category, cv=5)
    print("Accuracy: %0.5f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Loading Data

In [84]:
df = pd.read_csv("newtrain.csv", low_memory=False)

In [85]:
df_test = pd.read_csv("newtest.csv", low_memory=False)

In [86]:
allText = ""
for i in df.Text:
    allText += i + '.'

In [87]:
f = open('allText.txt', 'w')

In [88]:
f.write(allText)
f.close()

## Testing functions from Text Analysis assignement on Yahoo questions text data

In [9]:
path = "allText.txt"

In [10]:
analysis = textAnalysis(path)

In [29]:
# tokenize sentences and words an run them through a POS tagger
sentences,tagged_sents = analysis.gen_corpus_prep()

# generate noun phrases and proper nouns
nps,nps_strings,fd_nps = analysis.chunker(sentences,tagged_sents)

# deduce synsets through lesk algorithm and calculate distance between synsets
final_list,hyponyms = analysis.lesk_comparison(nps,sentences)

# print final report
analysis.final_report(fd_nps,final_list,hyponyms)

Starting NP parsing.
Done with NP parsing.
Done with FD of NPs
Extracting nouns...
Done extracting nouns
Done running Lesk on nouns
Done creating FD on lemmas

Most frequent noun phrases:
---------------------------
.how do                 91   
br & gt                 27   
.what do                22   
my boyfriend            19   
a guy                   19   
my comput               16   
the world               15   
a girl                  14   
.whi do                 11   
.i am                   9    
.i want                 9    
my husband              9    


Most frequent lemmas from NPs and their hyponyms:
-------------------------------------------------
can             194      (containerful.n.01)
get             184      (return.n.11)
know            156      (knowing.n.01)
find            89       (act.n.02)
need            77       (psychological_feature.n.01)
want            70       (need.n.01)
people          58       (group.n.01)
someone         51       (organis

## Prepping data

In [16]:
cat_dict = dict()

In [17]:
cat_dict = {1:"Business&Finance",2:"Computers&Internet",3:"Entertainment&Music",4:"Family&Relationships",5:"Education&Reference",6:"Health",7:"Science&Mathematics"}

In [33]:
# creating feature for lengths of text and converting to pandas series
lengths = []
for i in df.Text.iteritems():
    lengths.append(len(i[1]))
lengths = pd.Series(lengths)

In [34]:
# appending lengths feature to dataframe
df['lengths'] = lengths

In [20]:
# aggregate all text
all_text = []
for i in df.Text:
    all_text.append(i)

In [21]:
# aggregate all text and tokenize it
tok_sent = []
for i in df.Text:
    tok_sent.append(nltk.word_tokenize(i))

In [22]:
# pos_tag every word
pos_sent = []
for i in tok_sent:
    pos_sent.append(nltk.pos_tag(i))

## Implementation and Testing of Strategies

### 1st Strategy: Count occurrence of every tag (17.6%)

In [121]:
# creating list with all tags from upenn_tagset
tagset = ['$', "''", '(', ')', ',', '--', '.', ':','CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']

In [122]:
# creating columns of zeroes for every possible tag
for i in tagset:
    df[i] = pd.Series(np.zeros(df.shape[0]))

In [123]:
# Tokenizing and tagging every question training set
tagged_questions = []
for i in df.Text:
    tagged_questions.append(nltk.pos_tag(nltk.word_tokenize(i)))

In [124]:
# Tokenizing and tagging every question for test set
tagged_questions_test = []
for i in df_test.Text:
    tagged_questions_test.append(nltk.pos_tag(nltk.word_tokenize(i)))

In [125]:
# This is taking WAAAY too long, expected runtime > 1 hour. Use sparce vector array instead
time_check = time.time()
for n,q in enumerate(tagged_questions[:10]): # for every tagged questios from 'df.Text'...
    for tag in tagset: # take the list of possible tags...
        for w in q: # for every tagged word in the question text
            if w[1] == tag: # check if the tag matches the one the word is tagged as
                df[tag][n] += 1 # if so, increase counter and update the dataframe
    if n+1 % 10 == 0:
        print("Done with first",n)
print(round((time.time() - time_check)/60, 2),"minutes")

0.34 minutes


/home/wenqin/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [126]:
finished()

In [127]:
# round up all tags in one list for train data
tags_only_train = []
for q in tagged_questions:
    temp = ""
    for w in q:
        temp += w[1]
        temp += " "
    tags_only_train.append(temp)

In [128]:
# round up all tags in one list for test data
tags_only_test = []
for q in tagged_questions_test:
    temp = ""
    for w in q:
        temp += w[1]
        temp += " "
    tags_only_test.append(temp)

In [129]:
finished()

In [130]:
vec = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=5, max_features=433)

In [131]:
# Train sparce vectorizing
arr_train_feature_sparse = vec.fit_transform(pd.Series(tags_only_train))
arr_train_feature = arr_train_feature_sparse.toarray()
arr_train_feature.shape

(2698, 433)

In [137]:
arr_train_feature

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [132]:
# Test sparce vectorizing
arr_test_feature_sparse = vec.fit_transform(pd.Series(tags_only_test))
arr_test_feature = arr_test_feature_sparse.toarray()
arr_test_feature.shape

(1874, 377)

In [133]:
''' Training logistic regression with features in sparce array (arr_train_feature) 
and labels (df.Category)'''
logreg = LogisticRegression()
logreg_model_train = logreg.fit(arr_train_feature, df.Category)

In [134]:
# predicting against train features in sparce array
logreg_predictions_train = logreg_model_train.predict(arr_train_feature)

In [135]:
# crossvalidating
crossvalidation(logreg, arr_train_feature, df.Category)

Accuracy: 0.30913 (+/- 0.02)


In [136]:
# automatically creates submission with whatever file name you pass as the argument
create_submission("submission_3.csv")

TypeError: create_submission() missing 1 required positional argument: 'prediction'

### Second strategy: simple vectorizer and logistic regression (20.9%)

In [341]:
# function to facilitate testing of variations in CountVecotrizer parameters (min_df, max_features, etc.)
def logPredict(vec, train_set, other_set, scoring, return_train=True):
    # Train-data sparce vectorizing
    arr_train_feature_sparse = vec.fit_transform(train_set.Text)
    arr_train_feature = arr_train_feature_sparse.toarray()
    print("arr_train_feature.shape:", arr_train_feature.shape)

    # Other data sparce vectorizing
    arr_other_feature_sparse = vec.transform(other_set.Text)
    arr_other_feature = arr_other_feature_sparse.toarray()
    print("arr_other_feature.shape:", arr_other_feature.shape,'\n')

    ''' Training model with features in sparce array (arr_train_feature) and labels'''
    log_reg = LogisticRegression()
    logreg_model_train = log_reg.fit(arr_train_feature, train_set.Category)

    # print out crossvalidation or scoring results
    if scoring == 'cv':
        crossvalidation(log_reg, arr_train_feature, train_set.Category)
    elif scoring == 'scoring': 
        logreg_predictions_other = logreg_model_train.predict(arr_other_feature)
        print(accuracy_score(other_set.Category, logreg_predictions_other))
    else:
        print("Can't recognize scoring parameter.")
    
    if return_train:
        # predicting against train features in sparce array
        logreg_predictions_train = logreg_model_train.predict(arr_train_feature)
        print('\nReturning predictions for train set...')
        return logreg_predictions_train
    else:
        print('Returning predictions for other (dev or test) set...')
        # predicting against dev features in sparce array
        logreg_predictions_other = logreg_model_train.predict(arr_other_feature)
        return logreg_predictions_other

In [63]:
vec = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=5, max_features=1000)

In [82]:
logreg_predictions_test = logPredict(vec)

arr_train_feature.shape: (2698, 640)
arr_test_feature.shape: (1874, 640)
Accuracy: 0.34875 (+/- 0.04)


In [53]:
logreg_predictions_test

array([1, 1, 1, ..., 1, 1, 1])

In [194]:
# adding Category column to test set
df_test['Category'] = pd.Series(logreg_predictions_test)

In [195]:
# automatically creates submission with whatever file name you pass as the argument
create_submission("submission_5.csv")

### Third strategy: bigram, trigram and tetragram vectorizer and logistic regression (22.6%)

In [68]:
# notice the ngram_range parameter has "2, 4", so ngrams range 2 to 4
vec = CountVectorizer(ngram_range=(2, 4), token_pattern=r'\b\w+\b', min_df=5, max_features=640)

In [69]:
logreg_predictions_test = logPredict(vec)

arr_train_feature.shape: (2698, 640)
arr_test_feature.shape: (1874, 640)
Accuracy: 0.73904 (+/- 0.02)


In [73]:
# Train-data sparce vectorizing
arr_train_feature_sparse = vec.fit_transform(df.Text)
arr_train_feature = arr_train_feature_sparse.toarray()
print("arr_train_feature.shape:", arr_train_feature.shape)

# Test-data sparce vectorizing
arr_test_feature_sparse = vec.fit_transform(df_test.Text)
arr_test_feature = arr_test_feature_sparse.toarray()
print("arr_test_feature.shape:", arr_test_feature.shape)

''' Training logistic regression with features in sparce array (arr_train_feature) 
and labels (df.Category)'''
logreg = LogisticRegression()
logreg_model_train = logreg.fit(arr_train_feature, df.Category)

# predicting against train features in sparce array
logreg_predictions_train = logreg_model_train.predict(arr_train_feature)

arr_train_feature.shape: (2698, 640)
arr_test_feature.shape: (1874, 640)


In [75]:
crossvalidation(logreg, arr_train_feature, df.Category)

Accuracy: 0.34875 (+/- 0.04)


### Fourth strategy: same vectorizer but with Naive Bayes (%)

In [271]:
# function to facilitate testing of variations in CountVecotrizer parameters (min_df, max_features, etc.)
def NBPredict(model, vec, train_set, other_set, scoring, num_neighbors = 1, gamma= 0.001):
    '''
    model -  LogisticRegression, MultinomialNB, KNeighborsClassifier, svm
    '''
    # Train-data sparce vectorizing
    arr_train_feature_sparse = vec.fit_transform(train_set.Text)
    arr_train_feature = arr_train_feature_sparse.toarray()
    print("arr_train_feature.shape:", arr_train_feature.shape)

    # Other data sparce vectorizing
    arr_other_feature_sparse = vec.transform(other_set.Text)
    arr_other_feature = arr_other_feature_sparse.toarray()
    print("arr_other_feature.shape:", arr_other_feature.shape,'\n')

    ''' Training model with features in sparce array (arr_train_feature) and labels (df_.Category)'''
    if model == KNeighborsClassifier:
        nb = model(n_neighbors = num_neighbors)
    elif model == svm:
        nb = model.SVC(gamma=gamma, C=100)
    else:
        nb = model()
   
    nb_model_train = nb.fit(arr_train_feature, train_set.Category)    

    # predicting against dev features in sparce array
    nb_predictions_other = nb_model_train.predict(arr_other_feature)
    
    # print out crossvalidation or scoring results
    if scoring == 'cv':
        crossvalidation(nb, arr_train_feature, train_set.Category)
    elif scoring == 'dev': 
        print(accuracy_score(other_set.Category, nb_predictions_other))
        
    return nb_predictions_other

In [90]:
# notice the ngram_range parameter has "2, 4", so ngrams range 2 to 4
vec = CountVectorizer(ngram_range=(2, 4), token_pattern=r'\b\w+\b', min_df=5, max_features=640)

In [92]:
pd.DataFrame(df.Category).join(pd.DataFrame(nb_train_predictions, columns=['Predictions'])).head()

NameError: name 'nb_train_predictions' is not defined

### Fifth strategy: Tf-idf vectorizer

In [217]:
tfidf_vec = TfidfVectorizer(min_df=1, max_features=1000)

In [223]:
tfid_test_predictions = logPredict(tfidf_vec)

arr_train_feature.shape: (2698, 1000)
arr_test_feature.shape: (1874, 1000)
Accuracy: 0.50072 (+/- 0.02)


In [219]:
NBPredict(tfidf_vec)

arr_train_feature.shape: (2698, 1000)
arr_test_feature.shape: (1874, 1000) 

Accuracy: 0.45546 (+/- 0.04)


array([3, 3, 2, ..., 1, 1, 4])

In [255]:
create_submission('submission_6.csv', tfid_test_predictions)

### Proper Nouns

**develop training, dev, testing sets**

In [236]:
random_index = np.random.permutation(df.index)
df_shuffled = df.ix[random_index]
df_shuffled.reset_index(drop=True, inplace=True)
df_shuffled[:5]

,Category,Text,$,'',(,),",",--,.,:,...,VBD,VBG,VBN,VBP,VBZ,WDT,WP,WP$,WRB,``
0,4,can you give me tips on how to get over an ex?,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6,waht are sources of vitamin d?,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,structural framework of internet?,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,how do i get an ip hider for my computer to he...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,when sign makers go on strike,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [237]:
rows, columns = df_shuffled.shape
print("Rows:", rows)
print("Columns:", columns)

Rows: 2698
Columns: 47


**80/20 split for training/ dev**

In [238]:
train_size = round(rows*.8)
dev_size   = round(rows*.2)

In [239]:
df_train = df_shuffled.loc[:train_size]
print('train:',df_train.shape)

df_dev = df_shuffled.loc[train_size:dev_size+train_size].reset_index(drop=True)
print('dev:',df_dev.shape)

train: (2159, 47)
dev: (540, 47)


##Implement voting system between models

In [266]:
def vote_winner(pair):
    dic = {}
    for i in range(len(pair)):
        num = pair[i]
        if num in dic:
            dic[num] += 1
        else:
            dic[num] = 1
    return sorted(dic.items(), key=operator.itemgetter(1), reverse=True)[0][0]
     

[3,
 4,
 1,
 2,
 1,
 6,
 6,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 3,
 1,
 2,
 2,
 1,
 4,
 1,
 1,
 1,
 3,
 1,
 1,
 7,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 6,
 1,
 1,
 2,
 5,
 1,
 1,
 4,
 1,
 2,
 2,
 1,
 1,
 3,
 2,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 7,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 2,
 4,
 2,
 1,
 3,
 1,
 7,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 4,
 1,
 1,
 1,
 1,
 6,
 1,
 1,
 1,
 3,
 4,
 7,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 2,
 1,
 1,
 2,
 5,
 7,
 4,
 5,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 7,
 7,
 6,
 1,
 1,
 1,
 1,
 2,
 6,
 1,
 3,
 4,
 1,
 2,
 3,
 1,
 3,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 3,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 6,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 3,
 2,
 6,
 1,
 1,
 1,
 4,
 2,
 5,
 1,
 1,
 3,
 3,
 1,
 3,
 1,
 3,
 1,
 5,
 3,
 1,
 1,
 7,
 7,
 1,
 1,
 2,
 1,
 1,
 1,
 3,
 1,
 1,
 5,
 1,
 1,
 2,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 7,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 2,
 4,
 2,
 1,
 3,
 1,
 2,
 3,
 1,
 2,


## counting # of "I"
**Training**

In [269]:
tok_sent_train = []
for i in df_train.Text:
    tok_sent_train.append(nltk.word_tokenize(i))
    
num_i_train = []
for i in tok_sent_train:
    count = 0
    for j in i:
        if j == "i":
            count += 1
    num_i_train.append(count)
    
# num_i = pd.Series(num_i)
# df_train['num_i'] = num_i

df_train_features = pd.DataFrame({'num_i': num_i_train})

#Error: cannot join the feature columns...

vec_count = tfidf_vec = TfidfVectorizer(min_df=1, max_features=1000)
arr_train_feature_sparse = vec_count.fit_transform(df_train.Text)
arr_train_feature = arr_train_feature_sparse.toarray()
df_train_features['count'] = arr_test_feature

ValueError: Length of values does not match length of index

In [98]:
df_train_features.head()

,num_i
0,0
1,0
2,1
3,0
4,0


**Dev**

In [99]:
tok_sent_dev = []
for i in df_dev.Text:
    tok_sent_dev.append(nltk.word_tokenize(i))
    
num_i_dev = []
for i in tok_sent_dev:
    count = 0
    for j in i:
        if j == "i":
            count += 1
    num_i_dev.append(count)
    
# num_i_dev = pd.Series(num_i)
# df_dev['num_i'] = num_i


df_dev_features = pd.DataFrame({'num_i': num_i_dev})

In [100]:
print(df_dev_features.head(),'\nShape:',df_dev_features.shape)

   num_i
0      0
1      0
2      0
3      1
4      0 
Shape: (540, 1)


In [101]:
nb = MultinomialNB()
nb_model = nb.fit(df_train_features, df_train.Category)

In [102]:
nb_predictions = nb_model.predict(df_dev_features)
nb_predictions[:10]
 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [103]:
print(len(df_dev.Category))
print(len(nb_predictions))

540
540


In [104]:
accuracy_score(df_dev.Category, nb_predictions)

0.28888888888888886

In [105]:
crossvalidation(nb_model, df_train_features, df_train.Category)

Accuracy: 0.28393 (+/- 0.00)


##TFIDF

In [113]:
tfidf_vec = TfidfVectorizer(min_df=1, max_features=1190, stop_words='english')

**test on dev set**

In [256]:
dev_predictions_nb = NBPredict(MultinomialNB, tfidf_vec, df_train, df_dev, 'dev')
dev_predictions_logreg = NBPredict(LogisticRegression, tfidf_vec, df_train, df_dev, 'dev')
dev_predictions_KNN = NBPredict(KNeighborsClassifier, tfidf_vec, df_train, df_dev, 'dev', num_neighbors=3)
dev_predictions_svm = NBPredict(svm, tfidf_vec, df_train, df_dev, 'cv', gamma=0.005)


arr_train_feature.shape: (2159, 1000)
arr_other_feature.shape: (540, 1000) 

0.42037037037
arr_train_feature.shape: (2159, 1000)
arr_other_feature.shape: (540, 1000) 

0.47037037037
arr_train_feature.shape: (2159, 1000)
arr_other_feature.shape: (540, 1000) 

0.3
arr_train_feature.shape: (2159, 1000)
arr_other_feature.shape: (540, 1000) 

Accuracy: 0.50256 (+/- 0.05)


In [264]:
voted_results_dev = [vote_winner(pair) for pair in zip(dev_predictions_logreg, dev_predictions_nb, dev_predictions_svm)]

In [265]:
accuracy_score(voted_results_dev, df_dev.Category)

0.46111111111111114

**submit test set to Kaggle**

In [254]:
test_predictions_nb = NBPredict(MultinomialNB, tfidf_vec, df, df_test, 'cv')

arr_train_feature.shape: (2698, 1000)
arr_other_feature.shape: (1874, 1000) 

Accuracy: 0.45546 (+/- 0.04)


In [115]:
create_submission("submission_7.csv", test_predictions)

In [255]:
test_predictions_logreg = NBPredict(LogisticRegression, tfidf_vec, df, df_test, 'cv')

arr_train_feature.shape: (2698, 1000)
arr_other_feature.shape: (1874, 1000) 

Accuracy: 0.50072 (+/- 0.02)


In [249]:
#this one needs tuning...
test_predictions_KNN = NBPredict(KNeighborsClassifier, tfidf_vec, df, df_test, 'cv', num_neighbors=3)

arr_train_feature.shape: (2698, 1000)
arr_other_feature.shape: (1874, 1000) 

Accuracy: 0.30539 (+/- 0.03)


In [225]:
# tried gamma = 0.001, 0.003, 0.005 and 0.007. 0.005 has the best result
test_predictions_svm = NBPredict(svm, tfidf_vec, df, df_test, 'cv', gamma=0.005)

arr_train_feature.shape: (2698, 1000)
arr_other_feature.shape: (1874, 1000) 

Accuracy: 0.50663 (+/- 0.03)
Returning predictions for other (dev or test) set...


In [363]:
create_submission("submission_8.csv", test_predictions_logreg)

### apply voting system between three models: logreg, nb, and svm 

In [270]:
voted_results = [vote_winner(pair) for pair in zip(test_predictions_logreg, test_predictions_nb, test_predictions_svm)]

In [267]:
create_submission("submission_9.csv", voted_results)